In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# Math manipulation
import numpy as np
import pandas as pd

# Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Image processing
import cv2

# Utilities
import os
import random
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical
import codecs, json 
import re

from albumentations import Compose
import albumentations as A
import random

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

#utils
from keras_tqdm import TQDMNotebookCallback

# Models
from keras import applications
from keras.optimizers import SGD, Adam

from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

In [ ]:
base = "/Users/megantabbutt/Desktop/Computer Science Classes/762_AdvancedDeepLearning/762_Project/"

In [ ]:
FLAG_DEBUG = False
FLAG_GENERATE_TABLE = True
FLAG_TRAIN_IN_PRELOAD = False

img_height = 256 #- these should already be done in pre-processing 
img_width = 256 #- these should already be done in pre-processing 
img_channels = 1

batch_size = 32

In [ ]:
label_names=['Human', 'Interact front', 'Interact lat', 'Interact vert', 'Crowded', 
             'Drink', 'Curiosity', 'Queue', 'Low visibility', 'Nothing']

In [ ]:
def generate_pointer_table (path_frames, path_labels): # MEGAN REFACTOR
    pt_table = []
    
    label_file = open(path_labels)
    label_data = json.load(label_file)
    
    for frame in os.listdir(path_frames):
        if frame[-4:] == ".jpg":
            key = frame.split(".")[0]
            pt_table.append([key, label_data[key]])
        
    return pt_table

In [ ]:
def do_pointer_table():
    pointer_table_path_test = path_labels_base + 'pointer_table_test.json'
    pointer_table_shuffled_path_test = path_labels_base + 'pointer_table_shuffled_test.json'
    if FLAG_DEBUG: print ('Number of files: ', len(list_of_videos))

    if FLAG_GENERATE_TABLE:      
        pt_table = generate_pointer_table(path_frames_test, path_labels)
        json.dump(pt_table, codecs.open(pointer_table_path_test, 'w', encoding='utf-8'))
        print('Test Pointer table saved')
        random.shuffle(pt_table)
        json.dump(pt_table, codecs.open(pointer_table_shuffled_path_test, 'w', encoding='utf-8'))
        print('Test Shuffled pointer table saved')
        pt_table_test = np.array(pt_table)

    test_max_index = n_test
    print(len(pt_table_test))

    gc.collect()
    
    return pt_table_test

In [ ]:
augmentation_train = A.Compose([
    A.RandomContrast(limit=0.3, p=0.5),
    A.RandomGamma(gamma_limit=(70, 130), p=0.5),
    A.RandomBrightness(limit=0.6, p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.05, 
        scale_limit=0.05, 
        rotate_limit=10, 
        border_mode=cv2.BORDER_CONSTANT, 
        p=0.5), 
    A.ToFloat(max_value=255),
])

augmentation_test = A.Compose([
    A.ToFloat(max_value=255)
])

In [ ]:
def frame_from_pointer (path, video, frame, verbose=False):
    if verbose: print ('In '+path+'/'+str(video)+'.mp4' + ', taking frame ' + str(frame))
    cap = cv2.VideoCapture(path+'/'+str(video)+'.mp4')
    if (cap.isOpened()== False):
        print("Error opening video file") 
        return -1
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
    ret, frame = cap.read() # Capture next frame
    if ret==True:
        cap.release()
        return frame
    else:
        print("Error opening frame") 
        return -1

In [ ]:
def frame_from_pointer(path, vidframe):
    image = cv2.imread(path + vidframe + ".jpg")
    
    if image.shape[2] > 1: # grayscale got saved as three copies of the same image in three channels so just take one TODO - MEGAN FIX
        image, G2, G3 = cv2.split(image)
        
    return image

In [ ]:
def image_generator_lazy(pt_table, batch_size, augmentation, frame_path):
    
    while True:          # Select files (paths/indices) for the batch
        pointer_batch = pt_table[np.random.choice(pt_table.shape[0], batch_size, replace=False), :]
        batch_X = []
        batch_y = []
        
        for pointer in pointer_batch:
            frame = frame_from_pointer(path=frame_path, vidframe=pointer[0])
            frame = frame.astype(np.float32) # MT added 
            frame = augmentation(image=frame)['image']
            frame = frame.reshape(frame.shape[0], frame.shape[1], img_channels)
            label = to_categorical(pointer[1], num_classes=10)
        
            batch_X.append(frame)
            batch_y.append(label)
             
        # Return a tuple of (input,output) to feed the network          
        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        yield(batch_X, batch_y)

In [ ]:
def plot_cm(y_true, y_pred, labels_x, labels_y, figsize=(10,10)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax, linewidths=.5, xticklabels=labels_x, yticklabels=labels_y)
    ax.tick_params(axis='x', rotation=+25)
    ax.tick_params(axis='y', rotation=0)
    
    

# 1. Unbalanced Video Data from Paper

In [ ]:
path_frames_test = base + "Data/Exp1_frames_unbalanced_256/test/"

path_labels_base = base + "Data/Exp1_frames_unbalanced_256/labels_paper/"  
path_labels = path_labels_base + "labels_paper.json"

model_path = base + "ModelSave/Exp1_V2_model.h5"
acc_path = base + "ModelSave/Exp1_V2_accuracies.json"
n_test = len(os.listdir(path_frames_test))
n_test

In [ ]:
pt_table_test = do_pointer_table()

model = load_model(model_path) # load the best model saved in training phase

# Make predictions based on the test images
test_generator = image_generator_lazy(pt_table_test, batch_size=batch_size, augmentation=augmentation_test, frame_path=path_frames_test)

X_test = []
y_test = []
predictions = []
for i in range(0, int(len(pt_table_test)/batch_size)):
    X_batch, y_batch = next(test_generator)
    predictions_batch = model.predict_on_batch(X_batch)
    X_test.extend(X_batch)
    y_test.extend(y_batch)
    predictions.extend(predictions_batch)

X_test = np.array(X_test)
y_test = np.array(y_test)
predictions = np.array(predictions)

_, acc = model.evaluate(X_test, y_test)
print('Test acc:', acc)

In [ ]:
predictions = model.predict(X_test)
np.array(predictions).shape

In [ ]:
y_predict_singleval = np.array([np.argmax(xi) for xi in predictions])
y_true_singleval = np.array([np.argmax(xi) for xi in y_test])

cm = confusion_matrix(y_true_singleval, y_predict_singleval)

FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

F1 = 2*TP / (2*TP + FP + FN)
PREC = TP / (TP + FP)
REC = TP / (TP + FN)

FP, FN, TP, TN

In [ ]:
print(PREC[0], PREC[-1])
print(REC[0], REC[-1])
print(F1[0], F1[-1])

In [ ]:
plot_cm(np.array([np.argmax(xi) for xi in y_test]), np.array([np.argmax(xi) for xi in predictions]), 
        label_names[1:], label_names[1:], figsize=(9, 9))
_n=0

In [ ]:
with open(path_labels_base + 'pointer_table_shuffled_test.json') as fp:
    shuffled_test = json.load(fp)